# Домашнее задание 3
### Линейная и логистическая регрессия

Оформление дз:
Присылайте выполненное на почту `mlphystech@gmail.com` файлы .pdf и .ipynb или ссылку на свой гитхаб.

**Дедлайн: 11 апреля 23:59**

*Зуева Надежда*

ФИВТ МФТИ

Это задание будет посвящено линейной и логистической регрессии.
Мы надеемся, что вы уже знаете, что такое `kaggle`, так как сегодня нам предстоит поработать с самым *классическим* датасетом машинного обучения, а именно $-$ с Титаником!


Цель соревнования $-$ предсказать, кто из пассажиров выживет, а кто, увы, нет.

https://www.kaggle.com/c/titanic

## Теоретические вопросы
### 2 балла (по 0.5 за каждый верный ответ)
1. Что такое линейные модели классификации и регрессии? В чем их основные свойства? Где хорошо работают?
2. Что такое регуляризация и как она используется? 
3. Что такое MSE? Приведите пример.
4. Что такое градиентный спуск? Опишите базовый алгоритм работы.

P.S. просьба не копипастить из Википедии. Это все легко вычисляется и будет караться :)

## Задание 1 (4 балла)
Скачайте файлы `train.csv` и `test.csv`.

Выведите первые 10 строк из `train.csv`

In [1]:
import pandas
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import KFold

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#считайте данные из файла train.csv
data = pd.read_csv("train.csv")
#выведите первые 10 строк из data
data.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


Разобьем выборку на `X` и `y`, где для`y` будем строить предсказания

In [3]:
y = pd.Series(data['Survived'])
X = deepcopy(data).drop(["Survived"], axis=1)

Как мы видим, не все данные *полны*. Например, это видно в столбце `Age`. 
В качестве простейшего решения мы предлагаем заменить пропущенные значения на медианные (функция `numpy.median`)
Вы можете воспользоваться нашим методом или произвести собственный анализ и заменить значения более умным способом (profit: бонусные баллы).

Заменим отсутствующие значения возраста не просто на медиану тренировочного набора, а на медиану набора всех пассажиров (train + test)

In [4]:
data_test = pd.read_csv("test.csv")
print(set(data.keys()) - set(data_test.keys()))

{'Survived'}


In [5]:
data_all = pd.concat([data.drop(["Survived"], axis=1), data_test])
print(data.shape, data_test.shape, data_all.shape)

(891, 12) (418, 11) (1309, 11)


In [6]:
# Replace all the missing ages in the data with the median age
X["Age"] = X["Age"].fillna(data_all["Age"].median())
X.head(7)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


Теперь займемся **оцифровкой признаков**

Кстати, напомните, а зачем нам это нужно? :)

$*$ ваш ответ $*$

Какой параметр здесь бинарный? Оцифруйте его при помощи нулей и единиц.

In [7]:
X.loc[X["Sex"] == "male", "Sex"] = 1 #тут что-то должно быть
X.loc[X["Sex"] == "female", "Sex"] = 0 #тут что-то должно быть
X.head(7)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",1,28.0,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,E46,S


Выведите описание нашего датасета после изменений с помощью функции `describe`.
Сколько процентов пассажиров выжило? Каков был средний возраст всех пассажиров?

In [8]:
print (X.describe())

       PassengerId      Pclass         Sex         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    2.308642    0.647587   29.361582    0.523008   
std     257.353842    0.836071    0.477990   13.019697    1.102743   
min       1.000000    1.000000    0.000000    0.420000    0.000000   
25%     223.500000    2.000000    0.000000   22.000000    0.000000   
50%     446.000000    3.000000    1.000000   28.000000    0.000000   
75%     668.500000    3.000000    1.000000   35.000000    1.000000   
max     891.000000    3.000000    1.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


Продолжим заполнять пропуски. Рассмотрим графу пунктов отправления `Embarked` 

Там есть типы данных:

C = Cherbourg, Q = Queenstown, S = Southampton, а также `nan`

In [9]:
print(X["Embarked"].unique())

['S' 'C' 'Q' nan]


In [10]:
import collections
print(collections.Counter(X['Embarked']))

Counter({'S': 644, 'C': 168, 'Q': 77, nan: 2})


С учетом последней ячейки заполните пропущенные значения по своему усмотрению. Объясните свой выбор, затем оцифруйте признак `Embarked` 

Например, заменив все на {0,1,2} или {12,42,10}, как хотите :)

In [11]:
X["Embarked"] = X["Embarked"].fillna("S")
print(collections.Counter(X["Embarked"]))

Counter({'S': 646, 'C': 168, 'Q': 77})


In [12]:
#тут ваш код c оцифровкой 
encoders = {"Embarked": preprocessing.LabelEncoder()}
X["Embarked"] = encoders["Embarked"].fit_transform(X["Embarked"])
print(encoders["Embarked"].classes_)
X.head(7)


['C' 'Q' 'S']


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2
4,5,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2
5,6,3,"Moran, Mr. James",1,28.0,0,0,330877,8.4583,NaN,1
6,7,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,E46,2


Посмотрим на оставшиеся признаки

In [13]:
unique_val = {}
for column in X:
    unique_val[column] = X[column].unique().shape[0]
print("The number of unique items: ", unique_val)

The number of unique items:  {'PassengerId': 891, 'Pclass': 3, 'Name': 891, 'Sex': 2, 'Age': 88, 'SibSp': 7, 'Parch': 7, 'Ticket': 681, 'Fare': 248, 'Cabin': 148, 'Embarked': 3}


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null int64
dtypes: float64(2), int64(6), object(3)
memory usage: 76.6+ KB


- Удалим из тренировочного набора имена, они индивидуальны для каждого пассажира и при прямой оцифровке будут дублировать PassengerId
- Оцифруем фичу Cabin. Несмотря на ее отсутствие у более чем трех четвертей, она группирует некоторых людей (уникальных значений 147, а заполнено 204).
- По той же причине оцифруем фичу Ticket

In [15]:
X = X.drop(["Name"], axis=1)

In [16]:
encoders["Ticket"] = preprocessing.LabelEncoder()
X["Ticket"] = encoders["Ticket"].fit_transform(X["Ticket"])
encoders["Cabin"] = preprocessing.LabelEncoder()
X["Cabin"] = encoders["Cabin"].fit_transform(X["Cabin"].fillna("cabin info is absent"))

In [17]:
for feature in encoders:
    print(feature, encoders[feature].classes_[:10])

Embarked ['C' 'Q' 'S']
Ticket ['110152' '110413' '110465' '110564' '110813' '111240' '111320' '111361'
 '111369' '111426']
Cabin ['A10' 'A14' 'A16' 'A19' 'A20' 'A23' 'A24' 'A26' 'A31' 'A32']


In [18]:
X.head(7)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,1,22.0,1,0,523,7.2500,147,2
1,2,1,0,38.0,1,0,596,71.2833,81,0
2,3,3,0,26.0,0,0,669,7.9250,147,2
3,4,1,0,35.0,1,0,49,53.1000,55,2
4,5,3,1,35.0,0,0,472,8.0500,147,2
5,6,3,1,28.0,0,0,275,8.4583,147,1
6,7,1,1,54.0,0,0,85,51.8625,129,2


Каким образом можно отобрать фичи, которые будут влиять на наш скор? Тут нам помогут ваш жизненный опыт и ` Recursive Feature Elimination` из библиотеки `Sklearn` 

In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
model = LinearRegression()#тут можно подбирать параметры для повышения скора)
rfe_fit = RFE(model, 7).fit(X, y)
print(rfe_fit.ranking_)
#тут ваш код

[4 1 1 1 1 1 3 2 1 1]


Проверьте, похожи ли отобранные вами фичи на те, что представленны ниже?

In [20]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
print(features)
print(list(X.keys()[rfe_fit.support_]))

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked']


Выполним предсказания:

In [21]:
kfold = KFold(X.shape[0], n_folds=3, random_state=0)

In [22]:
predicts = []
for train_index, test_index in kfold:
    #тут ваш код
    model.fit(X.iloc[train_index], y[train_index])
    predicts.append(model.predict(X.iloc[test_index]))
predicts = np.concatenate(predicts)

In [23]:
print(predicts)

[ 1.13759079e-01  1.01557110e+00  6.08575842e-01  9.63545033e-01
  7.93640477e-02  1.70342379e-01  3.77434428e-01  1.29370917e-01
  5.48814717e-01  9.41788009e-01  6.62440799e-01  8.71079855e-01
  1.54591585e-01 -1.31267413e-01  6.80324001e-01  6.61531273e-01
  1.43191723e-01  3.11337399e-01  5.67155095e-01  7.09740998e-01
  2.78319783e-01  2.92912350e-01  7.26897998e-01  5.51639288e-01
  5.87292574e-01  3.63490356e-01  2.19664527e-01  4.70621068e-01
  6.58750890e-01  1.18820686e-01  5.10863951e-01  1.09885421e+00
  6.58338323e-01  9.68988494e-02  5.54658853e-01  4.02379070e-01
  2.18304619e-01  1.49021828e-01  6.02541003e-01  7.51242262e-01
  5.09309074e-01  7.79764309e-01  2.10871824e-01  9.13128686e-01
  7.04859423e-01  1.08243032e-01  1.31578650e-01  6.65827925e-01
  1.59089955e-01  6.30342499e-01  8.01151813e-02  1.47993052e-01
  9.50341308e-01  7.62785429e-01  4.20819298e-01  5.27145242e-01
  8.19644016e-01  2.14210247e-01  8.07777619e-01 -1.67370193e-02
  2.48582385e-01  9.95792

In [24]:
predicts[predicts > 0.5], predicts[predicts < 0.5] = 1,0

accuracy = np.count_nonzero(y == predicts)/y.count()
print ('Скор для линейной регрессии:',accuracy)

Скор для линейной регрессии: 0.7833894500561167


## Задание 2 (2 балла)
Теперь попробуем проделать то же самое для **логистической регрессии**

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [25]:
model = LogisticRegression()#тут можно подбирать параметры для повышения скора)

Вычислим оценку точности для всех кросс-валидационных фолдов: возвращаем массив баллов из 3-х фолдов

In [26]:
#тут ваш код
scores = []
for train_index, test_index in kfold:
    #тут ваш код
    model.fit(X.iloc[train_index], y[train_index])
    scores.append(model.score(X.iloc[test_index], y.iloc[test_index]))

print("Скор для логистической регрессии: " + str(np.array(scores).mean()))

Скор для логистической регрессии: 0.7912457912457912


###### Как бы вы объяснили прирост качества в логистической регрессии? 
Сможете побить наш бейзлайн? :)

## Задание 3 (1.5 балла)
Отправьте ваше решение в тестирующую систему и посмотрите, какое место вы займете

рассмотрим `test.csv`

In [ ]:
df_test = pandas.read_csv("test.csv")
print(df_test.describe())

Надо вернуть оцифрованные данные к первоначальному виду:

In [ ]:
#тут ваш код по возвращению оцифрованных параметров в первоначальный вид

df_test["Fare"] = df_test["Fare"].fillna(df_test["Fare"].median())

model.fit(#тут ваш код)
predicts = model.predict(#тут ваш код)

submission = pandas.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": predicts
})

submission.to_csv("submission.csv", index=False) #вот это отправляется в каггл

Можете попробовать и дальше "поиграться" с параметрами, чтобы подняться в лидерборде. Например, попробовать подобрать параметры в регрессорах. 
Удачи!
    Напоминаем, что справку по любой функции (в том числе и по регрессорам) можно получить, если с наведенным курсором на функцию нажать `Tab`+`Shift`